In [ ]:
import sys
sys.path.append("./notebooks")
sys.path.append("./src")

import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller

In [ ]:
data = np.load("../DATA/dataMonthly.npy")

### band indices:
| # | Index | Meaning (High Values ~ +1) | Meaning (Low / Negative Values ~ 0 → -1) |
|---|--------|-----------------------------|-------------------------------------------|
| 0 | **SAVI** (Soil-Adjusted Vegetation Index) | Dense, healthy vegetation; strong vigor | Bare soil, sparse or no vegetation |
| 1 | **EVI** (Enhanced Vegetation Index) | Dense, healthy canopy; reduced soil/atmosphere noise | Sparse or stressed vegetation, barren land |
| 2 | **NDRE705** (Red-Edge 705) | High chlorophyll; early healthy growth | Early stress; chlorophyll reduction begins |
| 3 | **NDRE740** (Red-Edge 740) | Balanced vegetation health | Moderate stress; photosynthetic decline |
| 4 | **NDRE783** (Red-Edge 783) | Very healthy vegetation; high biomass | Heavy stress or senescence |
| 5 | **NDVI** (Normalized Difference Vegetation Index) | Lush, green vegetation; high photosynthesis | Bare soil (~0), water or dead vegetation (<0) |
| 6 | **NDWI (Gao)** | High leaf water content; well-watered canopy | Water stress; dry or senescent leaves |
| 7 | **NDWI (McFeeters)** | Open water bodies (lakes, rivers) | Land or built-up areas; vegetation |
| 8 | **NBR** (Normalized Burn Ratio) | Healthy, unburned vegetation | Burned/disturbed areas; ash or bare soil |


In [ ]:
indices ={
    0: "SAVI",
    1: "EVI",
    2: "NDRE705",
    3: "NDRE740",
    4: "NDRE783",
    5: "NDVI",
    6: "NDWI GAO",
    7: "NDWI McFeeters",
    8: "NBR"
}

In [ ]:
import copy

def fill_zeros_1d(y):
    x = np.arange(len(y))
    zero_mask = y == 0
    if np.any(zero_mask) and np.any(~zero_mask):
        y = y.copy()
        y[zero_mask] = np.interp(x[zero_mask], x[~zero_mask], y[~zero_mask])
    return y

data_m = np.apply_along_axis(fill_zeros_1d, axis=0, arr=copy.deepcopy(data))

## Band analysis for single Pixel

In [ ]:
y = 250
x = 500
example_pixel_m = data_m[:, :, y, x]
example_pixel = data[:, :, y, x]

In [ ]:
x = np.arange(0, 69)
y = pd.Series(example_pixel_m[:, 1])
yd = y.diff(12)
y_nm = pd.Series(example_pixel[:, 1])

fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y_nm, name="Raw band")),
fig.add_trace(go.Scatter(x=x, y=y, name="Corrected Band"))
fig.add_trace(go.Scatter(x=x, y=yd, name="Differentiated Corrected Band"))
fig.update_layout(title_text="Visualization of band correction and differentiation")
fig.show()

This visualization shows the different time series processing steps on the example of a randomly selected pixel and the Enhanced Vegetation Index (EVI).  
Cloud-related gaps are being filled using an interpolation.  
The differentiation with a lag of 12 is used to deseasonalize the time series and make changes in the index more apparent.  

Further evaluation is needed to determine for which indices this workflow is suitable, as not every index exhibits seasonality. In such cases, a simple differentiation may suffice.

## Band Analysis over complete AOI

The plots contain the p-value of the Augmented-Dickey-Fuller-Test for stationarity for the cloud-corrected series

In [ ]:
result = data_m.mean(axis=(2, 3))
for i in range(9):
    x = np.arange(0, 69)
    y = pd.Series(result[:, i])
    y_d = y.diff(12)
    
    test_result = adfuller(y)
    
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=x, y=y, name="Corrected"))
    fig.add_trace(go.Scatter(x=x, y=y_d, name="Differentiated"))
    fig.update_layout(title_text=f"{indices[i]} p-value: {test_result[1]}",
                      width=800,
                      height=300)
    fig.show()

Those plots indicate, that most indices don't show seasonality and already are stationary and thus the differenciation wouldn't have a significant effect.  
Plots which should be de-seasonalized:
* SAVI
* EVI

In [ ]:
# p-values of differentiation

evi = pd.Series(result[:, 0]).diff(12).dropna()
savi = pd.Series(result[:, 1]).diff(12).dropna()

evi_result = adfuller(evi)
savi_result = adfuller(savi)

print(f"p-value of EVI: {evi_result[1]}")
print(f"p-value of SAVI: {savi_result[1]}")

Those p-values now suggest that differenciating with lag 12 is a reasonable step to ensure the series are stationary for further feature extraction

## AOI Band Visualization

In [ ]:
vegetation_index = data[67, 6, :, :]

In [ ]:
fig, axes = plt.subplots(3, 3, figsize=(10, 9))

for i, ax in enumerate(axes.flat):
    im = ax.imshow(data_m[68, i, :, :], cmap='YlGn', vmin=-1, vmax=1)
    ax.set_title(indices[i])
plt.tight_layout()

cbar = fig.colorbar(im, ax=axes.ravel().tolist(), shrink=0.4, label='Vegetation Index')
fig.set_size_inches(18, 10)
